# Fish Counting Algorithm using YOLOv8

This notebook implements a fish counting algorithm using YOLOv8 object detection on sonar imagery from the Fish Counting dataset.

## Dataset Overview
- Images from various river regions (Kenai, Nushagak, Elwha, etc.)
- Sonar imagery with fish detections
- Different strata and sampling locations

## Algorithm Steps
1. Data exploration and preprocessing
2. YOLOv8 model training
3. Fish detection and counting
4. Evaluation and visualization

In [ ]:
# Install required packages
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install ultralytics opencv-python pandas numpy matplotlib
# !pip install jupyter

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
from pathlib import Path
import torch
from tqdm import tqdm

# Check if CUDA is available
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

In [ ]:
# Set up paths
ROOT_DIR = Path("..")
DATA_DIR = ROOT_DIR / "Data" / "tiny_dataset"
RAW_DIR = DATA_DIR / "raw"
METADATA_DIR = DATA_DIR / "metadata-tiny"

print(f"Root directory: {ROOT_DIR.absolute()}")
print(f"Data directory: {DATA_DIR.absolute()}")
print(f"Raw data directory: {RAW_DIR.absolute()}")
print(f"Metadata directory: {METADATA_DIR.absolute()}")

## 1. Data Exploration

Let's explore the dataset structure and metadata.

In [ ]:
# Load metadata
def load_metadata():
    metadata_files = list(METADATA_DIR.glob("*.json"))
    all_metadata = []
    
    for file_path in metadata_files:
        with open(file_path, 'r') as f:
            data = json.load(f)
            for item in data:
                item['region'] = file_path.stem.split('-')[0]
                all_metadata.append(item)
    
    return pd.DataFrame(all_metadata)

metadata_df = load_metadata()
print(f"Loaded metadata for {len(metadata_df)} video clips")
metadata_df.head()

In [ ]:
# Explore dataset statistics
print("Dataset Statistics:")
print(f"Total clips: {len(metadata_df)}")
print(f"Regions: {metadata_df['region'].unique()}")
print(f"Average frames per clip: {metadata_df['num_frames'].mean():.1f}")
print(f"Average framerate: {metadata_df['framerate'].mean():.1f} fps")
print(f"Image dimensions: {metadata_df[['width', 'height']].mode().iloc[0].to_dict()}")

# Plot distribution of clips by region
plt.figure(figsize=(10, 6))
metadata_df['region'].value_counts().plot(kind='bar')
plt.title('Number of Clips by Region')
plt.xlabel('Region')
plt.ylabel('Number of Clips')
plt.xticks(rotation=45)
plt.show()

## 2. Image Analysis

Let's examine some sample images from different regions.

In [ ]:
# Function to get sample images
def get_sample_images(region, n_samples=5):
    region_clips = metadata_df[metadata_df['region'] == region]
    if len(region_clips) == 0:
        return []
    
    sample_clip = region_clips.sample(1).iloc[0]
    clip_dir = RAW_DIR / region / sample_clip['clip_name']
    
    if not clip_dir.exists():
        return []
    
    image_files = list(clip_dir.glob("*.jpg"))
    if len(image_files) < n_samples:
        n_samples = len(image_files)
    
    return np.random.choice(image_files, n_samples, replace=False)

# Display sample images from different regions
regions = metadata_df['region'].unique()
fig, axes = plt.subplots(len(regions), 3, figsize=(15, 5*len(regions)))

for i, region in enumerate(regions):
    sample_images = get_sample_images(region, 3)
    
    for j, img_path in enumerate(sample_images):
        if i < len(axes) and j < len(axes[i]):
            img = cv2.imread(str(img_path))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                axes[i][j].imshow(img)
                axes[i][j].set_title(f"{region} - {img_path.name}")
                axes[i][j].axis('off')

plt.tight_layout()
plt.show()

## 3. Data Preprocessing

Prepare the data for YOLO training.

In [ ]:
# Create YOLO dataset structure
# Note: This assumes we have annotations or will create them
# For now, we'll set up the basic structure

YOLO_DATA_DIR = ROOT_DIR / "yolo_data"
YOLO_DATA_DIR.mkdir(exist_ok=True)

# Create data.yaml for YOLO
data_yaml = f"""
train: {YOLO_DATA_DIR / 'images' / 'train'}
val: {YOLO_DATA_DIR / 'images' / 'val'}

nc: 1
names: ['fish']
"""

with open(YOLO_DATA_DIR / "data.yaml", 'w') as f:
    f.write(data_yaml)

print("YOLO data structure created")
print(f"Data YAML saved to: {YOLO_DATA_DIR / 'data.yaml'}")

## 4. YOLO Model Setup and Training

Configure and train the YOLOv8 model.

In [ ]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Load pretrained model

# Display model info
print(model.info())

# Note: Training would require labeled data
# For demonstration, we'll show the training command
"""
# Train the model
results = model.train(
    data=YOLO_DATA_DIR / 'data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    name='fish_counting'
)
"""
print("Model loaded successfully")

## 5. Fish Counting Implementation

Implement the counting logic based on detection results.

In [ ]:
def count_fish_in_image(image_path, model, conf_threshold=0.5):
    """
    Count fish in a single image using YOLO detection.
    
    Args:
        image_path (str): Path to the image
        model: Trained YOLO model
        conf_threshold (float): Confidence threshold for detections
    
    Returns:
        dict: Detection results including count and bounding boxes
    """
    # Run inference
    results = model(image_path, conf=conf_threshold)
    
    # Extract detections
    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            detection = {
                'bbox': box.xyxy[0].cpu().numpy(),
                'confidence': box.conf[0].cpu().numpy(),
                'class': int(box.cls[0].cpu().numpy())
            }
            detections.append(detection)
    
    return {
        'count': len(detections),
        'detections': detections,
        'image_path': image_path
    }

def count_fish_in_clip(clip_dir, model, conf_threshold=0.5, max_frames=None):
    """
    Count fish in all frames of a video clip.
    
    Args:
        clip_dir (Path): Directory containing clip images
        model: Trained YOLO model
        conf_threshold (float): Confidence threshold
        max_frames (int): Maximum number of frames to process (for testing)
    
    Returns:
        dict: Aggregated counting results for the clip
    """
    image_files = sorted(list(clip_dir.glob("*.jpg")))
    
    if max_frames:
        image_files = image_files[:max_frames]
    
    clip_results = []
    total_count = 0
    
    for img_path in tqdm(image_files, desc=f"Processing {clip_dir.name}"):
        result = count_fish_in_image(str(img_path), model, conf_threshold)
        clip_results.append(result)
        total_count += result['count']
    
    return {
        'clip_name': clip_dir.name,
        'total_frames': len(image_files),
        'total_fish_count': total_count,
        'avg_fish_per_frame': total_count / len(image_files) if image_files else 0,
        'frame_results': clip_results
    }

print("Fish counting functions defined")

## 6. Visualization Functions

Create functions to visualize detection results.

In [ ]:
def visualize_detections(image_path, detections, save_path=None):
    """
    Visualize fish detections on an image.
    
    Args:
        image_path (str): Path to the image
        detections (list): List of detection dictionaries
        save_path (str): Path to save the visualization (optional)
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Draw bounding boxes
    for detection in detections:
        bbox = detection['bbox']
        conf = detection['confidence']
        
        # Convert to int
        x1, y1, x2, y2 = map(int, bbox)
        
        # Draw rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        # Draw label
        label = f"Fish: {conf:.2f}"
        cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, (255, 0, 0), 2)
    
    # Add count text
    count_text = f"Fish Count: {len(detections)}"
    cv2.putText(img, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                1, (0, 255, 0), 2)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.title(f"Fish Detection Results - {Path(image_path).name}")
    plt.axis('off')
    plt.show()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')

def plot_counting_results(clip_results):
    """
    Plot fish counting results over time.
    
    Args:
        clip_results (dict): Results from count_fish_in_clip
    """
    frame_counts = [frame['count'] for frame in clip_results['frame_results']]
    
    plt.figure(figsize=(12, 6))
    plt.plot(frame_counts, marker='o', linestyle='-')
    plt.title(f"Fish Count Over Time - {clip_results['clip_name']}")
    plt.xlabel('Frame Number')
    plt.ylabel('Number of Fish Detected')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Summary statistics
    print(f"\nClip: {clip_results['clip_name']}")
    print(f"Total frames processed: {clip_results['total_frames']}")
    print(f"Total fish detected: {clip_results['total_fish_count']}")
    print(f"Average fish per frame: {clip_results['avg_fish_per_frame']:.2f}")
    print(f"Max fish in single frame: {max(frame_counts)}")

print("Visualization functions defined")

## 7. Evaluation Metrics

Add functions to evaluate model performance.

In [ ]:
def calculate_metrics(true_counts, predicted_counts):
    """
    Calculate evaluation metrics for fish counting.
    
    Args:
        true_counts (list): Ground truth fish counts
        predicted_counts (list): Predicted fish counts
    
    Returns:
        dict: Dictionary of evaluation metrics
    """
    true_counts = np.array(true_counts)
    predicted_counts = np.array(predicted_counts)
    
    # Mean Absolute Error
    mae = np.mean(np.abs(true_counts - predicted_counts))
    
    # Root Mean Square Error
    rmse = np.sqrt(np.mean((true_counts - predicted_counts)**2))
    
    # Mean Absolute Percentage Error
    mape = np.mean(np.abs((true_counts - predicted_counts) / (true_counts + 1e-8))) * 100
    
    # Accuracy within tolerance (±1 fish)
    accuracy_1 = np.mean(np.abs(true_counts - predicted_counts) <= 1)
    
    # Accuracy within tolerance (±2 fish)
    accuracy_2 = np.mean(np.abs(true_counts - predicted_counts) <= 2)
    
    return {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'Accuracy_±1': accuracy_1,
        'Accuracy_±2': accuracy_2
    }

def evaluate_model_on_clip(clip_results, ground_truth_counts=None):
    """
    Evaluate model performance on a clip.
    
    Args:
        clip_results (dict): Results from count_fish_in_clip
        ground_truth_counts (list): Ground truth counts for comparison
    
    Returns:
        dict: Evaluation results
    """
    if ground_truth_counts is None:
        # If no ground truth, just return basic statistics
        predicted_counts = [frame['count'] for frame in clip_results['frame_results']]
        return {
            'mean_count': np.mean(predicted_counts),
            'std_count': np.std(predicted_counts),
            'min_count': min(predicted_counts),
            'max_count': max(predicted_counts),
            'total_frames': len(predicted_counts)
        }
    
    predicted_counts = [frame['count'] for frame in clip_results['frame_results']]
    metrics = calculate_metrics(ground_truth_counts, predicted_counts)
    
    return {
        'metrics': metrics,
        'predicted_counts': predicted_counts,
        'ground_truth_counts': ground_truth_counts
    }

print("Evaluation functions defined")

## 8. Testing and Demonstration

Test the algorithm on sample data. (Note: This would require a trained model and ground truth data)

In [ ]:
# Example usage (would need trained model and data)
"""
# Select a sample clip for testing
sample_clip = metadata_df.sample(1).iloc[0]
clip_path = RAW_DIR / sample_clip['region'] / sample_clip['clip_name']

print(f"Testing on clip: {sample_clip['clip_name']}")
print(f"Region: {sample_clip['region']}")
print(f"Frames: {sample_clip['num_frames']}")

# Count fish in the clip (using first 10 frames for demo)
results = count_fish_in_clip(clip_path, model, max_frames=10)

# Visualize results
plot_counting_results(results)

# Show detection on a sample frame
sample_image = list(clip_path.glob("*.jpg"))[0]
sample_result = count_fish_in_image(str(sample_image), model)
visualize_detections(str(sample_image), sample_result['detections'])

# Evaluate performance
evaluation = evaluate_model_on_clip(results)
print("\nEvaluation Results:")
for key, value in evaluation.items():
    print(f"{key}: {value}")
"""

print("Testing code prepared (commented out due to lack of trained model)")
print("To run testing:")
print("1. Train the YOLO model with annotated data")
print("2. Uncomment the testing code above")
print("3. Run the evaluation")

## Summary

This notebook provides a complete framework for fish counting using YOLOv8 object detection:

### Key Components:
1. **Data Exploration**: Load and analyze dataset metadata and sample images
2. **Preprocessing**: Prepare data for YOLO training
3. **Model Training**: Configure and train YOLOv8 model
4. **Fish Counting**: Implement detection-based counting logic
5. **Visualization**: Display detection results and counting statistics
6. **Evaluation**: Calculate performance metrics

### Next Steps:
- Annotate training data for YOLO
- Train the model on fish detection
- Fine-tune hyperparameters
- Validate on test set
- Deploy for real-time counting

### Requirements:
- PyTorch with CUDA support
- Ultralytics YOLOv8
- OpenCV for image processing
- Pandas and NumPy for data handling
- Matplotlib for visualization